In [37]:
import kagglehub
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import random

## dataset load

In [24]:
# netflix
netflix = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/Netflix.csv'
netflix = pd.read_csv(netflix)
netflix.shape

(15860, 8)

In [25]:
#amazon
amazon = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/amazon.csv'
amazon = pd.read_csv(amazon)
amazon.shape

(52390, 8)

In [26]:
#apple
apple = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/apple.csv'
apple = pd.read_csv(apple)
apple.shape

(14711, 8)

In [27]:
#hbo
hbo = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/hbo.csv'
hbo = pd.read_csv(hbo)
hbo.shape

(7967, 8)

In [28]:
#hulu
hulu = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/hulu.csv'
hulu = pd.read_csv(hulu)
hulu.shape

(8342, 8)

## data pre-process

In [29]:
imdb_df = pd.concat([netflix, amazon, apple, hbo, hulu], ignore_index=True)
imdb_df = imdb_df.drop_duplicates()
imdb_df = imdb_df.dropna(subset=['imdbNumVotes'])

In [32]:
Z = 1.96  # Z-score for 95% confidence level
p = 0.5   # Estimated proportion (maximum variability)
e = 0.05  # Margin of error
N = imdb_df['imdbNumVotes']
imdb_df['post_count'] = np.ceil((Z**2 * p * (1 - p)) / (e**2 + (Z**2 * p * (1 - p)) / N))

In [40]:
imdb_df = imdb_df.sort_values(by='imdbNumVotes', ascending=False)
imdb_df

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,post_count
16014,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,"DK, FI, JP, NL, NO, SE",385.0
90981,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,JP,385.0
71,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,"AD, BA, BE, BG, CA, CV, CZ, ES, GQ, HR, HU, IN...",385.0
83057,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,"AD, AR, BE, BO, BR, BZ, CL, CO, CR, DK, DO, EC...",385.0
51,The Dark Knight,movie,"Action, Crime, Drama",2008.0,tt0468569,9.0,2938846.0,"AD, AT, AU, BA, BG, CA, CH, CZ, DE, DK, ES, FI...",385.0
...,...,...,...,...,...,...,...,...,...
52416,My Yiddish Papi,movie,"Animation, Short, War",2017.0,tt7970448,6.4,5.0,"AG, AU, BB, BM, BS, BZ, CA, CM, GB, GH, GI, GY...",5.0
67012,Summer & Todd - L'allegra fattoria,tv,Animation,2021.0,tt18135286,8.4,5.0,"AD, ES, IT, SM, VA",5.0
15421,"The Cheery Lee, Village Headman",tv,"Comedy, Drama",2023.0,tt28364262,6.6,5.0,"ID, MY, PH, SG, TH",5.0
11117,The G Word,movie,"Comedy, Short",2016.0,tt5758674,5.0,5.0,TW,5.0


## Fetch posts

In [36]:
## X
import snscrape.modules.twitter as sntwitter
import random
import pandas as pd

# 假设 imdb_df 已加载并包含以下列：title, type, genres, releaseYear, imdbId, imdbAverageRating, imdbNumVotes, availableCountries, post_count
# 例如： imdb_df = pd.read_csv('your_data.csv')

# 创建一个列表，用于存储最终的结果
result_data = []

# 遍历每一行，根据电影标题查询推文
for index, row in imdb_df.iterrows():
    query = row['title']  # 使用电影标题作为搜索关键词
    max_samples = int(row['post_count'])  # 从 post_count 列获取随机抽取数量
    tweets = []

    # 使用 SNScrape 搜索推文
    for tweet in sntwitter.TwitterSearchScraper(f"{query}").get_items():
        tweets.append(tweet.content)  # 保存推文内容
        if len(tweets) >= 1000:  # 限制最大抓取数量为 1000 条
            break

    # 检查抓取的推文数量是否足够
    if len(tweets) >= max_samples:
        sampled_tweets = random.sample(tweets, max_samples)  # 随机抽取指定数量的推文
    else:
        sampled_tweets = tweets  # 如果抓取的推文不足 max_samples，直接使用所有抓取的推文

    # 将结果保存到字典中，并加入到结果列表
    result_data.append({
        'title': row['title'],
        'type': row['type'],
        'genres': row['genres'],
        'releaseYear': row['releaseYear'],
        'imdbId': row['imdbId'],
        'imdbAverageRating': row['imdbAverageRating'],
        'imdbNumVotes': row['imdbNumVotes'],
        'availableCountries': row['availableCountries'],
        'post_count': max_samples,
        'sampled_tweets': sampled_tweets
    })

# 将结果转换为 DataFrame
result_df = pd.DataFrame(result_data)

# 显示结果
print(result_df.head())

